# Imports

In [ ]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns
import string, random

# make

In [ ]:
!cd .. && make clean && make all
##!rm -r instancias && mkdir instancias

## Definitions

In [ ]:
BIN = "./../build/ContarPalabras"
TEST_FILE = "../data/corpus"
runs = []

In [ ]:
def correr_experimento(t_leer, t_max, files):
    # Leer archivo de la instancia.
    # Crear proceso para ejecutar el codigo.

    process = subprocess.Popen([BIN] + [t_leer, t_max] + files, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    
    # Correr experimento.
    exit_code = process.wait()

    time_l, time_max, _ = tuple(process.stderr.read().split("\n"))
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [float(time_l), float(time_max)]

## txt generator: a agus le gusta que le digan

In [ ]:
def gen_words_s(filename, char, n):
    file = open(f'instancias/{filename}.txt', 'a')
    for _ in range(n):
        word = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(3, 9)))
        word = char + word
        file.write(word + '\n')
    file.close()

In [ ]:
def txt(txtname, size):
    file = open(f'instancias/{txtname}.txt', 'w+')
    for _ in range(size):
        word = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(3, 9)))
        file.write(word + '\n')
    

for x in range(20 + 1):
    txt(x, 5000)

In [ ]:
correr_experimento('1', '1', ["0.txt"])

## Todos los tipos

In [ ]:
cantArchivos = 8
archivos = []
for _ in range(cantArchivos):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')

for threadsLectura in range(1, cantArchivos + 1):
    for threadsMax in range(1, 26 + 1):
        runs.append([str(threadsLectura), str(threadsMax), archivos, '1'])

# experimento : threadsMaximo

In [ ]:
cantArchivos = 8
archivos = []
for _ in range(cantArchivos):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')

for threadsMax in range(1, 27):
    runs.append(['1', str(threadsMax), archivos, '2'])
print(runs)    

# experimento : threadsLectura

In [ ]:
cantArchivos = 20
archivos = []
for _ in range(cantArchivos):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')

for threadsLectura in range(1, cantArchivos + 1):
    runs.append([str(threadsLectura), '1', archivos, '3'])

# Experimento peor caso, distribucion uniforme

In [ ]:
archivos = []
archivos.append('instancias/uniforme.txt')



for i in range(20):
    runs.append(['1',str(i+1),archivos,'7'])


# Experimento peor caso, distribucion exponencial

In [ ]:
archivos = []
archivos.append('instancias/semi-normal.txt')



for i in range(20):
    runs.append(['1',str(i+1),archivos,'8'])

    

In [ ]:
archivos = []
archivos.append('instancias/exponencial.txt')



for i in range(20):
    runs.append(['1',str(i+1),archivos,'9'])

    

In [ ]:
print(runs)

# variando la cantidad de archivos por cantidad de threads:

In [ ]:
archivos = []
for cantArchivos in range(1 , 13 + 1):
    archivos.append(f'instancias/{random.randint(1, 20)}.txt')
    for threadsLectura in range(1, cantArchivos + 1):
        runs.append([str(threadsLectura), '1', archivos.copy(), '4'])

In [ ]:
run = runs[0]
print(run)
correr_experimento(run[0], run[1], run[2])

### Ejectuar los experimentos:

In [ ]:
# [cantThreads][cantFiles][cantThreadsMax][TIEMPO_MAX][TIEMPO_LEER]
columnas = ["threadsLectura", "threadsMaximo", "cantArchivos", "TIEMPO_MAX", "TIEMPO_CARGAR", "EXPERIMENTO"];
filas = [];
numero = 1
T = 50 #Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for run in runs:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(runs)))
    numero += 1
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiemposMax = []
    tiemposCargar = []
    tiempo = []
    #print(experimento[4], experimento[5])
    for _ in range(0, T):
        tiempo = correr_experimento(run[0], run[1], run[2])
        tiemposCargar.append(tiempo[0])
        tiemposMax.append(tiempo[1])
    tiempoCargar = np.median(tiemposCargar);
    tiempoMax = np.median(tiemposMax);
    filas.append([run[0], run[1], len(run[2]), tiempoMax, tiempoCargar, run[3]]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultadoDistribuciones.csv", index=False, header=True);